In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import string
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RACHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('Suspicious Communication on Social Platforms.csv')
df

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1
...,...,...
19996,I dont. But what is complaining about it goi...,0
19997,Bahah yeah i&;m totally just gonna&; get pis...,0
19998,hahahahaha >:) im evil mwahahahahahahahahaha,0
19999,What&;s something unique about Ohio? :),0


Dataset Source: https://www.kaggle.com/code/cahyareza/trip-advisor-reviews/input

#### Data Preprocessing 

In [ ]:
def parse_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)   
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))    
    # Tokenize text into words
    tokens = word_tokenize(text)   
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]    
    # Join tokens back into text
    parsed_text = ' '.join(filtered_tokens)    
    return parsed_text

In [ ]:
nltk.download('stopwords')
df['comments'] = df['comments'].apply(parse_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RACHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
X = df['comments']
y = df['tagging']

In [ ]:
# Tokenize texts
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index

In [ ]:
# Pad sequences
max_length = 100
df1 = pad_sequences(sequences, maxlen=max_length)

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df1,y,test_size=0.2)

In [ ]:
from tensorflow.keras.layers import Dropout
# Define model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=max_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
250/250 [==============================] - 26s 94ms/step - loss: 0.5889 - accuracy: 0.6774 - val_loss: 0.5005 - val_accuracy: 0.7456
Epoch 2/10
250/250 [==============================] - 23s 93ms/step - loss: 0.4193 - accuracy: 0.8070 - val_loss: 0.4478 - val_accuracy: 0.7921
Epoch 3/10
250/250 [==============================] - 23s 94ms/step - loss: 0.3379 - accuracy: 0.8588 - val_loss: 0.4312 - val_accuracy: 0.8118
Epoch 4/10
250/250 [==============================] - 24s 95ms/step - loss: 0.2807 - accuracy: 0.8882 - val_loss: 0.4221 - val_accuracy: 0.8220
Epoch 5/10
250/250 [==============================] - 24s 95ms/step - loss: 0.2302 - accuracy: 0.9136 - val_loss: 0.4110 - val_accuracy: 0.8403
Epoch 6/10
250/250 [==============================] - 24s 94ms/step - loss: 0.1894 - accuracy: 0.9321 - val_loss: 0.4366 - val_accuracy: 0.8458
Epoch 7/10
250/250 [==============================] - 26s 106ms/step - loss: 0.1637 - accuracy: 0.9412 - val_loss: 0.4879 - val_accuracy

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

126/126 [==============================] - 3s 23ms/step - loss: 0.4882 - accuracy: 0.8613
Test loss: 0.4881916344165802
Test accuracy: 0.8612846732139587
